In [4]:
force_include = ['abc', 'ksfjl', 'jiow']
print("reference_viruses['h3n2']=reference_viruses['h3n2']+"+str(force_include))

reference_viruses['h3n2']=reference_viruses['h3n2']+['abc', 'ksfjl', 'jiow']


In [81]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [82]:
with open('augur_output/flu_seasonal_h3n2_ha_6y_tree.json', 'r') as jsonfile:
    data_6y = json.load(jsonfile)

with open('augur_output/flu_seasonal_h3n2_ha_12y_tree.json', 'r') as jsonfile:
    data_12y = json.load(jsonfile)

with open('augur_output/flu_seasonal_h3n2_ha_12y_tree_titers.json', 'r') as jsonfile:
    data_12y_titers = json.load(jsonfile)

In [83]:
print(data_12y_titers['attr'].keys())

[u'num_date', u'num_date_confidence', u'region_confidence', u'region', u'age_score', u'region_entropy', u'ne', u'rb', u'glyc', u'num_gender', u'avg_age', u'dTiter', u'cTiterSub', u'lbi', u'cTiter', u'div', u'age', u'ep', u'clade_membership', u'dTiterSub']


In [84]:
#Pull out mutations at tree tips
tip_muts = {}

def traverse(branch):
    if 'children' not in branch.keys():
        tip_muts[branch['strain']]=[branch['aa_muts']['HA1'], branch['aa_muts']['HA2'], 
                                           branch['aa_muts']['SigPep'],branch['attr']['num_date'], 
                                           branch['attr']['clade_membership']]
    else:
        for child in branch['children']:
            traverse(child)


traverse(data_12y)

In [85]:
#Make dictionary into a pandas df
df = pd.DataFrame(tip_muts).T
df.reset_index(inplace=True)
df.rename(columns={'index': 'strain',0: "HA1_muts", 1: "HA2_muts", 2: "SigPep_muts", 3: "date", 4: "clade"}, inplace=True)
df['passage'] = np.select((df.strain.str.contains('egg'), df.strain.str.contains('cell')), ('egg', 'cell'))

df.head(20)

,strain,HA1_muts,HA2_muts,SigPep_muts,date,clade,passage
0,A/Abobo/GR628/2012-cell,[],[],[],2012.42,unassigned,cell
1,A/Acores/SU43/2012-egg,[],[],[A16T],2012.85,unassigned,egg
2,A/Acre/15093/2010,[N38Y],[],[],2010.17,unassigned,0
3,A/Acre/26954/2010,[I223V],[],[],2010.25,unassigned,0
4,A/Adana/117/2014-cell,[],[],[],2014.02,3c3,cell
5,A/Adjame/GR765/2011-cell,[],[I149M],[],2011.7,unassigned,cell
6,A/Afghanistan/2950/2009-cell,[S91G],[],[],2009.78,unassigned,cell
7,A/Afghanistan/622/2017,[],"[K139R, L198I]",[],2017.9,A1b/135K,0
8,A/Afghanistan/AF1169/2008,[R307I],[F3L],[],2008.13,unassigned,0
9,A/Aichi/115/2008-cell,[D53G],[],[],2008.65,unassigned,cell


In [86]:
#make tidy version of df where each mutation gets a row
mut_df = pd.DataFrame(columns=['mutation']+ list(df.columns))

count=0
for i, r in df.iterrows():

    for ha1_mut in r['HA1_muts']:
        mut_df.loc[count]= ['HA1'+str(ha1_mut)] + list(df.loc[i])
        count+=1
        
    for ha2_mut in r['HA2_muts']:
        mut_df.loc[count]= ['HA2'+str(ha2_mut)] + list(df.loc[i])
        count+=1
        
    for sp_mut in r['SigPep_muts']:
        mut_df.loc[count]= ['SP'+str(sp_mut)] + list(df.loc[i])
        count+=1


In [97]:
mut_df.loc[(mut_df.clade=='3c2.A') & (mut_df.mutation=='HA1T160K')]
# print(len(mut_df.loc[(mut_df.clade=='3c2.A') & (mut_df.passage=='cell')]))

,mutation,strain,HA1_muts,HA2_muts,SigPep_muts,date,clade,passage
180,HA1T160K,A/Cambodia/Y1218307/2014-cell,[T160K],[],[],2014.917180,3c2.A,cell
265,HA1T160K,A/Dnipro/768/2016-cell,"[T160K, G263E]",[],[],2016.320329,3c2.A,cell
301,HA1T160K,A/Fiji/60/2016-egg,"[N121T, T160K, L194P, D225G]",[],[],2016.402464,3c2.A,egg
567,HA1T160K,A/Jiangxi-Zhushan/1560/2016-cell,"[S91N, T160K]",[R216K],[],2016.539357,3c2.A,cell
841,HA1T160K,A/Nebraska/19/2015-egg,"[G78D, K92R, T160K, L194P]",[],[],2015.807666,3c2.A,egg
1228,HA1T160K,A/Singapore/GP1523/2014-egg,"[T160K, L194P, D225N]",[],[],2014.520192,3c2.A,egg
1255,HA1T160K,A/SouthAfrica/2982/2015-egg,"[T160K, L194P]",[],[],2015.421629,3c2.A,egg
1276,HA1T160K,A/SouthAustralia/21/2015-egg,"[T10M, T160K, L194P, T203I]",[],[F15S],2015.194387,3c2.A,egg
1819,HA1T160K,A/Wisconsin/66/2015-cell,[T160K],[],[],2015.301164,3c2.A,cell


In [38]:
top_muts = {}
for pas_type in mut_df['passage'].unique():
    top = mut_df[mut_df.passage==pas_type].groupby('mutation')['mutation'].count().sort_values(ascending=False)[:10]
    top_muts[pas_type] = list((g_name, g) for g_name, g in top.iteritems())

total_cell = len(mut_df[mut_df.passage=='cell'])
total_egg = len(mut_df[mut_df.passage=='egg'])
print('Number of strains with this mutation \n(out of '+ str(total_cell)+ ' cell-passaged strains and '+ str(total_egg) + ' egg-passaged strains)')
print(pd.DataFrame(top_muts))

Number of strains with this mutation 
(out of 642 cell-passaged strains and 609 egg-passaged strains)
               0            cell             egg
0  (HA1V323I, 5)  (HA1T160K, 13)  (HA1G186V, 52)
1   (HA1D53N, 5)  (HA2A201V, 11)  (HA1L194P, 51)
2  (HA2N116S, 5)  (HA1R261Q, 11)  (HA1T160K, 23)
3  (HA1I140M, 4)   (HA2V18M, 10)  (HA1D225G, 18)
4   (HA2A43S, 4)    (HA1D53N, 9)  (HA1S219F, 17)
5  (HA1S219F, 3)   (HA1N158K, 9)  (HA1T203I, 14)
6  (HA2I186V, 3)     (SPA16T, 9)  (HA1S219Y, 11)
7   (HA1I25V, 3)   (HA1N225D, 8)   (HA1H156R, 9)
8   (HA1R33Q, 3)   (HA1P221L, 8)   (HA1A138S, 9)
9  (HA1F193S, 3)   (HA1V112I, 8)   (HA1H156Q, 8)


In [88]:
print(mut_df[mut_df.clade=='3c3.A']['HA1_muts'])

80                                            [S159Y]
160                                             [L3F]
174                                           [K207R]
175                                           [K207R]
176                                           [K207R]
177                               [D53N, K83E, Q311R]
178                               [D53N, K83E, Q311R]
179                               [D53N, K83E, Q311R]
186                                                []
215                                             [L3I]
267                                           [S265G]
295                                       [L3F, I48M]
296                                       [L3F, I48M]
335                                           [A128T]
369                                           [F193L]
417     [L3I, I25V, E62G, I140T, S143P, F193S, T328D]
418     [L3I, I25V, E62G, I140T, S143P, F193S, T328D]
419     [L3I, I25V, E62G, I140T, S143P, F193S, T328D]
420     [L3I, I25V, E62G, I1